- 2) [Genepop](http://kimura.univ-montp2.fr/~rousset/Genepop.htm)

In [ ]:
!wget http://hapmap.ncbi.nlm.nih.gov/downloads/genotypes/hapmap3/plink_format/draft_2/relationships_w_pops_121708.txt

- metadata (인종정보 추가 파일)

In [1]:
! head -4 relationships_w_pops_121708.txt

FID	IID	dad	mom	sex	pheno	population
2357	NA19625	0	0	2	0	ASW
2367	NA19702	NA19700	NA19701	1	0	ASW
2367	NA19700	0	0	1	0	ASW


In [12]:
from collections import defaultdict

In [13]:
f = open('relationships_w_pops_121708.txt')
pop_ind = defaultdict(list)
f.readline()  # header
for line in f:
    toks = line.rstrip().split('\t')
    fam_id = toks[0]
    ind_id = toks[1]
    pop = toks[-1]
    pop_ind[pop].append((fam_id, ind_id))
f.close()

In [14]:
for pop in pop_ind:
    print pop, pop_ind[pop][:3]

YRI [('Y004', 'NA18500'), ('Y004', 'NA18501'), ('Y004', 'NA18502')]
CHB [('NA18555', 'NA18555'), ('NA18524', 'NA18524'), ('NA18526', 'NA18526')]
CHD [('NA17962', 'NA17962'), ('NA17965', 'NA17965'), ('NA17966', 'NA17966')]
TSI [('NA20502', 'NA20502'), ('NA20504', 'NA20504'), ('NA20505', 'NA20505')]
MKK [('2563', 'NA21302'), ('2563', 'NA21301'), ('2563', 'NA21303')]
MEX [('M001', 'NA19650'), ('M001', 'NA19649'), ('M001', 'NA19648')]
LWK [('NA19027', 'NA19027'), ('NA19028', 'NA19028'), ('NA19031', 'NA19031')]
CEU [('1334', 'NA10846'), ('1334', 'NA12146'), ('1334', 'NA12239')]
ASW [('2357', 'NA19625'), ('2367', 'NA19702'), ('2367', 'NA19700')]
JPT [('NA18956', 'NA18956'), ('NA18940', 'NA18940'), ('NA18942', 'NA18942')]
GIH [('NA20845', 'NA20845'), ('NA20846', 'NA20846'), ('NA20847', 'NA20847')]


## Check for consistency issues
relationships_w_pops_121708.txt 와 hapmap1.ped의 정보 비교

In [13]:
! head -3 hapmap1.ped |cut -d' ' -f1-10

2427 NA19919 NA19908 NA19909 1 -9 G G C C
2431 NA19916 0 0 1 -9 G G C C
2424 NA19835 0 0 2 -9 G G C C


In [12]:
all_inds = []
for inds in pop_ind.values():
    all_inds.extend(inds)
    
for line in open('hapmap1.ped'):
    toks = line.rstrip().replace(' ', '\t').split('\t')
    fam = toks[0]
    ind = toks[1]
    if (fam, ind) not in all_inds:
        print('Problems with %s/%s' % (fam, ind))

Problems with 2469/NA20281


## 1) Convert from PLINK to Genepop


In [10]:
from genomics.popgen.plink.convert import to_genepop

to_genepop(PLINK prefix, Genepop prefix, Dictionary population, header="plink2gp")

- 이 함수는 consistency step에서 확인한 것과 같이 매칭되지 않는 ID를 가지는 경우와 population정보가 없는 경우를 모두 제외하고 작동함

In [15]:
to_genepop('hapmap1_auto', 'hapmap1_auto', pop_ind)
# to_genepop('hapmap10_auto_noofs_2', 'hapmap10_auto_noofs_2', pop_ind)
# hapmap1_auto.gp
# hapmap1_auto.pops

## 2-1) Using the in-memory parser
- .comment_line : header정보
- .loci_list : locus 정보
- .pop_list : population 정보
- .pupulations : 각 population과 그에 속하는 개인들의 genotype정보

In [1]:
from Bio.PopGen.GenePop import read

In [2]:
rec = read(open('hapmap1_auto.gp')) # Careful: this will severely increase your memory usage, consider it optional

In [7]:
print('Header: %s' % rec.comment_line)

Header: plink2gp


In [14]:
print(rec.loci_list[:5]) # loci

['1/rs9660180/1712891', '1/rs2840531/2276268', '1/rs3122923/2445360', '1/rs28688388/2576163', '1/rs3100865/2795967']


In [17]:
print('Population names: %s' % ', '.join(rec.pop_list))

Population names: 2436/NA19983, 1459/NA12865, NA18594/NA18594, NA18140/NA18140, NA20881/NA20881, NA19007/NA19007, NA19372/NA19372, M005/NA19652, 2581/NA21371, NA20757/NA20757, Y105/NA19099


In [50]:
ind = rec.populations[0][0]
print(ind[0], ind[1][:5])
# coding: ACGT - 1234

('2427/NA19919', [(4, 4), (2, 2), (2, 1), (4, 4), (3, 4)])


In [3]:
print('Header: %s' % len(rec.comment_line))
print('Number of loci %d' % len(rec.loci_list))
print('Number of populations %d' % len(rec.pop_list))
print('Population names: %s' % ', '.join(rec.pop_list))
print('Individuals per population %s' % ', '.join([str(len(inds)) for inds in rec.populations])) # last individual is used to identify a population
ind = rec.populations[1][0]
print('Individual %s, SNP %s, alleles: %d %d' % (ind[0], rec.loci_list[0], ind[1][0][0], ind[1][0][1]))
del rec # delete the record because it takes up a lot of memory

Header: 8
Number of loci 14032
Number of populations 11
Population names: 2436/NA19983, 1459/NA12865, NA18594/NA18594, NA18140/NA18140, NA20881/NA20881, NA19007/NA19007, NA19372/NA19372, M005/NA19652, 2581/NA21371, NA20757/NA20757, Y105/NA19099
Individuals per population 82, 165, 84, 85, 88, 86, 90, 77, 171, 88, 167
Individual 1328/NA06989, SNP 1/rs9660180/1712891, alleles: 4 4


## 2-2) Using the Large File Parser
- read_large

it worn't load the whole in-memory file, but provide an iterator instead

In [1]:
from Bio.PopGen.GenePop.LargeFileParser import read as read_large

In [3]:
def count_individuals(fname):
    rec = read_large(open(fname))
    pop_sizes = []
    for line in rec.data_generator():
        if line == ():  # 빈 tuple - marker of a new population
            pop_sizes.append(0)
        else:
            pop_sizes[-1] += 1
    return pop_sizes

In [4]:
print('Individuals per population %s' % ', '.join([str(ninds) for ninds in count_individuals('hapmap1_auto.gp')]))
# ()
#('2427/NA19919', [(4, 4), (2, 2), (2, 1), (4, 4), (3, 4), (1, 1), 
#                  (4, 4), (2, 2), (1, 1), (1, 4), (4, 4), (1, 1),
#                  (4, 4), (1, 1), (1, 4), (1, 4), (1, 1), (1, 4),...

Individuals per population 82, 165, 84, 85, 88, 86, 90, 77, 171, 88, 167
